# Chapter 5 - 일급 함수

파이썬의 함수는 일급 객체다.<br>
**일급 객체**는 다음과 같은 작업을 수행할 수 있는 프로그램 개체를 말한다.
- 런타임에 생성할 수 있다.
- 데이터 구조체의 변수나 요소에 할당할 수 있다.
- 함수 인수로 전달할 수 있다.
- 함수 결과로 반환할 수 있다.

## 함수를 객체처럼 다루기

In [1]:
def fact(n):
    '''return n factorial'''
    return 1 if n < 2 else n * fact(n - 1)


print(fact(42))
print(fact.__doc__)
print(type(fact))

1405006117752879898543142606244511569936384000000000
return n factorial
<class 'function'>


## 고위 함수

함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 **고위 함수**라고 한다.<br>
파이썬3의 경우 고위 함수의 예시로 map(), filter() 등이 있지만, 지능형 리스트와 제너레이터 표현식이 소개된 이후 이 함수들의 중요성은 떨어졌다.<br>

In [2]:
print(list(map(fact, filter(lambda n: n % 2, range(6)))))
print([fact(n) for n in range(6) if n % 2])  # 가독성이 더 좋다

[1, 6, 120]
[1, 6, 120]


## 일곱 가지 맛의 콜러블 객체

호출할 수 있는 객체인지 알아보려면 callable() 내장 함수를 사용하면 된다.<br>
파이썬 [데이터 모델 문서](https://docs.python.org/3/reference/datamodel.html)는 다음 일곱 가지 콜러블을 나열하고 있다.

|콜러블 타입|설명|
|-|-|
|사용자 정의 함수|def 문이나 람다 표현식으로 생성|
|내장 함수|len()이나 time.strftime()처럼 C 언어로 구현된 함수(CPython의 경우)|
|내장 메서드|dict.get()처럼 C언어로 구현된 메서드|
|메서드|클래스 본체에 정의된 함수|
|클래스|호출될 때 자신의 \_\_new\_\_() 메서드를 실행해서 객체를 생성하고, \_\_init\_\_()으로 초기화한 후, 최종적으로 호출자에 객체를 반환|
|클래스 객체|클래스가 \_\_call\_\_() 메서드를 구현하면 이 클래스의 객체는 함수로 호출 가능|
|제너레이터 함수|yield 키워드를 사용하는 함수나 메서드|

In [3]:
def f(a, *, b):
    return a, b


print(f(1, b=2))
# print(f(1, 2, 3))  # 에러!
print(f(b=3, a=5))

(1, 2)
(5, 3)


## 매개변수에 대한 정보 읽기

In [4]:
def clip(text, max_len=80):
    '''
    max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다
    '''
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    
    if end is None:  # 공백이 없다
        end = len(text)
        
    return text[:end].rstrip()


print(clip.__defaults__)  # 기본값
print(clip.__code__)
print(clip.__code__.co_varnames)  # 인수 및 지역변수 정보
print(clip.__code__.co_argcount)  # 인수 개수

(80,)
<code object clip at 0x7f4271215390, file "<ipython-input-4-a2a576bd79e1>", line 1>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


1. co_argcount를 통해 인수는 2개 있음을 알 수 있다.
2. co_varnames의 첫 2개인 text, max_len이 인수명임을 알 수 있다.
3. \_\_defaults\_\_를 통해 기본값 80이 있음을 알 수 있다.
4. 그리고 이 기본값은 마지막 인수인 max_len에 해당하는 것을 알 수 있다.

정보가 그다지 한눈에 들어오지 않는 것을 확인할 수 있다.<br>
하지만 inspect 모듈을 사용하면 더 깔끔하게 처리할 수 있다.

In [5]:
from inspect import signature

sig = signature(clip)
print(sig)

for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

(text, max_len=80)
POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


## 함수 애너테이션

```python
def clip(text, max_len=80):
```

위의 코드에 애너테이션을 추가한 버전은 다음과 같다.

```python
def clip(text:str, max_len:'int > 0'=80) -> str:
```

애너테이션은 전혀 처리하지 않으며, 단지 함수 객체 안의 dict형 __annotations__ 속성에 저장될 뿐이다.<br>
검사, 단속, 검증 등 아무런 행동을 취하지 않기 때문에 애너테이션은 파이썬 인터프리터에 아무런 의미가 없다.

## 함수형 프로그래밍을 위한 패키지

operator 모듈은 산술 연산자에 해당하는 여러 연산자를 함수화하여 제공한다.

In [6]:
from functools import reduce
from operator import mul


def fact(n):
    return reduce(mul, range(1, n + 1))


print(fact(10))

3628800


In [9]:
import operator

print([name for name in dir(operator) if not name.startswith('_')])

['abs', 'add', 'and_', 'attrgetter', 'concat', 'contains', 'countOf', 'delitem', 'eq', 'floordiv', 'ge', 'getitem', 'gt', 'iadd', 'iand', 'iconcat', 'ifloordiv', 'ilshift', 'imatmul', 'imod', 'imul', 'index', 'indexOf', 'inv', 'invert', 'ior', 'ipow', 'irshift', 'is_', 'is_not', 'isub', 'itemgetter', 'itruediv', 'ixor', 'le', 'length_hint', 'lshift', 'lt', 'matmul', 'methodcaller', 'mod', 'mul', 'ne', 'neg', 'not_', 'or_', 'pos', 'pow', 'rshift', 'setitem', 'sub', 'truediv', 'truth', 'xor']


functools.partial()로 인수를 고정할 수도 있다.

In [10]:
from functools import partial

triple = partial(mul, 3)
print(triple(7))  # mul(3, 7)
print(list(map(triple, range(1, 10))))

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]
